## Reading from bronze

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim,col

In [0]:
df = spark.table("bike_store.bonze_data.crm_cust_info")

In [0]:
df.display()

## Data Transformation


In [0]:
# Triming, removing spaces on strings
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
df.display()

In [0]:
# Normalization of the gender and marital status

df = (
    df.withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "M", "Married")
        .when(F.upper(F.col("cst_marital_status")) == "S", "Single")
        .otherwise("N/A")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "M", "Male")
        .when(F.upper(F.col("cst_gndr")) == "F", "Female")
        .otherwise("N/A")
    )
)


In [0]:
df.display()

In [0]:
# Renaming columns

RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "creation_date"
}

for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.display()

## Write into silver table 


In [0]:
(
    df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("bike_store.silver_data.crm_customer")
)

In [0]:
%sql
SELECT *
from bike_store.silver_data.crm_customer